In [1]:
import os
import warnings
import pandas as pd
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")
import tqdm

In [2]:
%load_ext autoreload
%autoreload 2
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl

In [3]:
SBL = StatsBombLoader(getter="remote", creds={"user": None, "passwd": None})

In [4]:
competitions2 = SBL.competitions()
selected_competitions = competitions2[(competitions2.competition_id == 43) & (competitions2.season_id == 106) |
                                        (competitions2.competition_id == 55) & (competitions2.season_id == 43) |
                                        (competitions2.competition_id == 53) & (competitions2.season_id == 106) |
                                        (competitions2.competition_id == 72) & (competitions2.season_id == 107)]


selected_competitions.head(10)

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
26,106,43,FIFA World Cup,International,male,2022
62,43,55,UEFA Euro,Europe,male,2020
64,106,53,UEFA Women's Euro,Europe,female,2022
65,107,72,Women's World Cup,International,female,2023


In [5]:
# Get games from all selected competitions
games = pd.concat([
    SBL.games(row.competition_id, row.season_id)
    for row in selected_competitions.itertuples()
])

In [6]:
games.head(2)

,game_id,season_id,competition_id,competition_stage,game_day,game_date,home_team_id,away_team_id,home_score,away_score,venue,referee
0,3857256,106,43,Group Stage,3,2022-12-02 21:00:00,786,773,2,3,Stadium 974,Fernando Andrés Rapallini
1,3869151,106,43,Round of 16,4,2022-12-03 21:00:00,779,792,2,1,Ahmad bin Ali Stadium,Szymon Marciniak


In [7]:
games_verbose = tqdm.tqdm(list(games.itertuples()), desc="Loading game data")
teams, players = [], []
actions = {}
for game in games_verbose:
    # load data
    teams.append(SBL.teams(game.game_id))
    players.append(SBL.players(game.game_id))
    events = SBL.events(game.game_id)
    # convert data
    actions[game.game_id] = spadl.statsbomb.convert_to_actions(events, game.home_team_id)

teams = pd.concat(teams).drop_duplicates(subset="team_id")
players = pd.concat(players)

Loading game data: 100%|██████████| 210/210 [04:14<00:00,  1.21s/it]


In [8]:
datafolder = "/home/msai/adnan002/repos/xB-360/data"

# Create data folder if it doesn't exist
if not os.path.exists(datafolder):
    os.mkdir(datafolder)
    print(f"Directory {datafolder} created.")

spadl_h5 = os.path.join(datafolder, "spadl-statsbomb.h5")

# Store all spadl data in h5-file
with pd.HDFStore(spadl_h5) as spadlstore:
    spadlstore["competitions"] = selected_competitions
    spadlstore["games"] = games
    spadlstore["teams"] = teams
    spadlstore["players"] = players[['player_id', 'player_name', 'nickname']].drop_duplicates(subset='player_id')
    spadlstore["player_games"] = players[['player_id', 'game_id', 'team_id', 'is_starter', 'starting_position_id', 'starting_position_name', 'minutes_played']]
    for game_id in actions.keys():
        spadlstore[f"actions/game_{game_id}"] = actions[game_id]

Directory /home/msai/adnan002/repos/xB-360/data created.


In [9]:
features_h5 = os.path.join(datafolder, "features.h5")
labels_h5 = os.path.join(datafolder, "labels.h5")

In [10]:
games = pd.read_hdf(spadl_h5, "games")
print("nb of games:", len(games))

nb of games: 210


In [11]:
import socceraction.spadl as spadl
import socceraction.vaep.features as fs
import socceraction.vaep.labels as lab

xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    fs.bodypart,
    fs.bodypart_onehot,
    fs.result,
    fs.result_onehot,
    fs.goalscore,
    fs.startlocation,
    fs.endlocation,
    fs.movement,
    fs.space_delta,
    fs.startpolar,
    fs.endpolar,
    fs.team,
    fs.time,
    fs.time_delta
]

for game in tqdm.tqdm(list(games.itertuples()), desc=f"Generating and storing features in {features_h5}"):
    actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")
    gamestates = fs.gamestates(spadl.add_names(actions), 3)
    gamestates = fs.play_left_to_right(gamestates, game.home_team_id)
    
    X = pd.concat([fn(gamestates) for fn in xfns], axis=1)
    X.to_hdf(features_h5, f"game_{game.game_id}")

Generating and storing features in /home/msai/adnan002/repos/xB-360/data/features.h5: 100%|██████████| 210/210 [00:34<00:00,  6.12it/s]


In [12]:
yfns = [lab.scores, lab.concedes, lab.goal_from_shot]

for game in tqdm.tqdm(list(games.itertuples()), desc=f"Computing and storing labels in {labels_h5}"):
    actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")   
    Y = pd.concat([fn(spadl.add_names(actions)) for fn in yfns], axis=1)
    Y.to_hdf(labels_h5, f"game_{game.game_id}")

Computing and storing labels in /home/msai/adnan002/repos/xB-360/data/labels.h5: 100%|██████████| 210/210 [00:27<00:00,  7.63it/s]


In [13]:
predictions_h5 = os.path.join(datafolder, "predictions.h5")

In [14]:
traingames = games
testgames = games

# 1. Select feature set X
xfns = [
    fs.actiontype,
    fs.actiontype_onehot,
    #fs.bodypart,
    fs.bodypart_onehot,
    fs.result,
    fs.result_onehot,
    fs.goalscore,
    fs.startlocation,
    fs.endlocation,
    fs.movement,
    fs.space_delta,
    fs.startpolar,
    fs.endpolar,
    fs.team,
    #fs.time,
    fs.time_delta,
    #fs.actiontype_result_onehot
]
nb_prev_actions = 1

Xcols = fs.feature_column_names(xfns, nb_prev_actions)

def getXY(games,Xcols):
    # generate the columns of the selected feature
    X = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting features"):
        Xi = pd.read_hdf(features_h5, f"game_{game_id}")
        X.append(Xi[Xcols])
    X = pd.concat(X).reset_index(drop=True)

    # 2. Select label Y
    Ycols = ["scores","concedes"]
    Y = []
    for game_id in tqdm.tqdm(games.game_id, desc="Selecting label"):
        Yi = pd.read_hdf(labels_h5, f"game_{game_id}")
        Y.append(Yi[Ycols])
    Y = pd.concat(Y).reset_index(drop=True)
    return X, Y

X, Y = getXY(traingames,Xcols)
print("X:", list(X.columns))
print("Y:", list(Y.columns))

Selecting label: 100%|██████████| 210/210 [00:01<00:00, 153.50it/s]

X: ['type_id_a0', 'type_pass_a0', 'type_cross_a0', 'type_throw_in_a0', 'type_freekick_crossed_a0', 'type_freekick_short_a0', 'type_corner_crossed_a0', 'type_corner_short_a0', 'type_take_on_a0', 'type_foul_a0', 'type_tackle_a0', 'type_interception_a0', 'type_shot_a0', 'type_shot_penalty_a0', 'type_shot_freekick_a0', 'type_keeper_save_a0', 'type_keeper_claim_a0', 'type_keeper_punch_a0', 'type_keeper_pick_up_a0', 'type_clearance_a0', 'type_bad_touch_a0', 'type_non_action_a0', 'type_dribble_a0', 'type_goalkick_a0', 'bodypart_foot_a0', 'bodypart_head_a0', 'bodypart_other_a0', 'bodypart_head/other_a0', 'result_id_a0', 'result_fail_a0', 'result_success_a0', 'result_offside_a0', 'result_owngoal_a0', 'result_yellow_card_a0', 'result_red_card_a0', 'goalscore_team', 'goalscore_opponent', 'goalscore_diff', 'start_x_a0', 'start_y_a0', 'end_x_a0', 'end_y_a0', 'dx_a0', 'dy_a0', 'movement_a0', 'start_dist_to_goal_a0', 'start_angle_to_goal_a0', 'end_dist_to_goal_a0', 'end_angle_to_goal_a0']
Y: ['scores

In [15]:
# 3. train classifiers F(X) = Y
import xgboost

Y_hat = pd.DataFrame()
models = {}
for col in list(Y.columns):
    model = xgboost.XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbosity=1)
    model.fit(X, Y[col])
    models[col] = model

In [16]:
from sklearn.metrics import brier_score_loss, roc_auc_score, log_loss

testX, testY = X, Y

def evaluate(y, y_hat):
    p = sum(y) / len(y)
    base = [p] * len(y)
    brier = brier_score_loss(y, y_hat)
    print(f"  Brier score: %.5f (%.5f)" % (brier, brier / brier_score_loss(y, base)))
    ll = log_loss(y, y_hat)
    print(f"  log loss score: %.5f (%.5f)" % (ll, ll / log_loss(y, base)))
    print(f"  ROC AUC: %.5f" % roc_auc_score(y, y_hat))

for col in testY.columns:
    Y_hat[col] = [p[1] for p in models[col].predict_proba(testX)]
    print(f"### Y: {col} ###")
    evaluate(testY[col], Y_hat[col])


### Y: scores ###
  Brier score: 0.00892 (0.82964)
  log loss score: 0.04513 (0.75268)
  ROC AUC: 0.84257
### Y: concedes ###
  Brier score: 0.00218 (0.88796)
  log loss score: 0.01300 (0.75246)
  ROC AUC: 0.87791


In [17]:
# get rows with game id per action
A = []
for game_id in tqdm.tqdm(games.game_id, "Loading game ids"):
    Ai = pd.read_hdf(spadl_h5, f"actions/game_{game_id}")
    A.append(Ai[["game_id"]])
A = pd.concat(A)
A = A.reset_index(drop=True)

# concatenate action game id rows with predictions and save per game
grouped_predictions = pd.concat([A, Y_hat], axis=1).groupby("game_id")
for k, df in tqdm.tqdm(grouped_predictions, desc="Saving predictions per game"):
    df = df.reset_index(drop=True)
    df[Y_hat.columns].to_hdf(predictions_h5, f"game_{int(k)}")

Loading game ids:   0%|          | 0/210 [00:00<?, ?it/s]

Saving predictions per game: 100%|██████████| 210/210 [00:09<00:00, 22.43it/s]


In [18]:
# show spadl_h5
with pd.HDFStore(spadl_h5) as spadlstore:
    print(spadlstore.keys())

# show features_h5
with pd.HDFStore(features_h5) as featuresstore:
    print(featuresstore.keys())

['/competitions', '/games', '/player_games', '/players', '/teams', '/actions/game_3788741', '/actions/game_3788742', '/actions/game_3788743', '/actions/game_3788744', '/actions/game_3788745', '/actions/game_3788746', '/actions/game_3788747', '/actions/game_3788748', '/actions/game_3788749', '/actions/game_3788750', '/actions/game_3788751', '/actions/game_3788752', '/actions/game_3788753', '/actions/game_3788754', '/actions/game_3788755', '/actions/game_3788756', '/actions/game_3788757', '/actions/game_3788758', '/actions/game_3788759', '/actions/game_3788760', '/actions/game_3788761', '/actions/game_3788762', '/actions/game_3788763', '/actions/game_3788764', '/actions/game_3788765', '/actions/game_3788766', '/actions/game_3788767', '/actions/game_3788768', '/actions/game_3788769', '/actions/game_3788770', '/actions/game_3788771', '/actions/game_3788772', '/actions/game_3788773', '/actions/game_3788774', '/actions/game_3788775', '/actions/game_3788776', '/actions/game_3794685', '/action